In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
# import seaborn as sns
import pickle
import gc
import time
from tqdm import tqdm

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [2]:
import pandas as pd
import lightgbm as lgb
import numpy as np
import datetime
from sklearn.feature_selection import SelectKBest,chi2
# from minepy import MINE
from tqdm import tqdm
import gc 
import time

## 特征列

In [3]:
origin = [
    'item_price_level', 'item_sales_level', 'item_collected_level',
    'item_pv_level', 'user_gender_id', 'user_age_level', 'user_occupation_id',
    'user_star_level', 'context_page_id', 'shop_review_num_level',
    'shop_review_positive_rate', 'shop_star_level', 'shop_score_service',
    'shop_score_delivery', 'shop_score_description'
]

features_zero = [
    'user_count_first_time_sub_full_time', 'user_item_day_num',
    'user_trade_hot', 'user_hour_query', 'yesterhour',
    'user_minute_count', 'item_occupation_trade_count', 'user_context_count',
    'user_sell_power_mean_item_minus', 'city_trade_hot', 'city_item_count_his',
    'user_count_first_time_sub', 'user_item_next_time_sub_full_time',
    'user_item_next_time_sub', 'item_property_split_count', 'city_item_count',
    'max_click', 'brand_trade_percent', 'user_item_num',
    'user_shop_next_time_sub_full_time', 'city_brand_count', 'item_trade_hot',
    'user_sell_power', 'user_sell_trade_power', 'brand_item_count',
    'item_trade_percent', 'user_hot', 'user_shop_count_first_time_sub',
    'user_item_day_num_full_time', 'is_last_click', 'item_minute_query',
    'user_is_his', 'shop_item_count_his', 'user_item_last_time_sub_full_time',
    'user_shop_next_time_sub', 'max_click_time_sub', 'user_item_count',
    'item_score', 'last_time_sub', 'user_sell_power_mean',
    'user_shop_last_time_sub', 'item_hot', 'user_sell_trade_power_mean',
    'user_sell_trade_power_mean_item_minus', 'last_time_sub_full_time',
    'brand_trade_hot', 'item_gender_trade_count',
    'next_time_sub_full_time', 'max_click_full_time',
    'user_shop_last_time_sub_full_time', 'user_item_hour_query',
    'shop_item_count', 'user_minute_query', 'brand_item_count_his',
    'item_age_trade_count', 'user_day_query', 'max_user_item_click_time_sub',
    'item_is_trade_his', 'item_pred_category_score_item%', 'user_query_day',
    'item_day_query', 'city_hot', 'user_item_count_first_time_sub',
    'item_score2', 'user_query_day_hour', 'hour', 'city_brand_count_his',
    'is_last_user_item_click', 'user_shop_num_full_time',
    'item_pred_category_score', 'user_item_day_query',
    'user_item_last_time_sub', 'user_is_trade_his',
    'user_shop_day_num_full_time', 'brand_hot', 'user_shop_day_num',
    'user_count', 'shop_trade_hot', 'is_last_user_item_click_full_time',
    'next_time_sub', 'user_shop_num', 'shop_hot',
    'user_shop_count_first_time_sub_full_time',
    'item_pred_property_score_item%',
    'user_item_count_first_time_sub_full_time', 'max_user_item_click',
    'max_user_item_click_time_sub_full_time', 'user_trade_percent',
    'item_hour_query', 'user_yesterday_query', 'min_time', 'user_query_minute',
    'max_user_item_click_full_time', 'is_last_click_full_time'
]

features_cjf = ['user_lastquery_timedelta','user_lastquery','item_price_level_shop_id_bayes_rate_2','item_price_level_shop_id_bayes_rate_1','item_price_stair_shop_id_bayes_rate_1','item_id_bayes_rate_1','item_price_stair_shop_id_bayes_rate_2','item_id_shop_id_bayes_rate_1','item_id_bayes_rate_2','item_id_shop_id_bayes_rate_2','item_id_shop_star_level_bayes_rate_1','item_city_id_shop_id_bayes_rate_2','item_id_shop_review_num_level_bayes_rate_2','item_id_shop_star_level_bayes_rate_2','item_sales_stair_shop_id_bayes_rate_2','item_city_id_shop_id_bayes_rate_1','item_sales_stair_shop_id_bayes_rate_1','shop_id_bayes_rate_2','item_id_shop_review_num_level_bayes_rate_1','item_pv_stair_shop_id_bayes_rate_2','item_brand_id_shop_id_bayes_rate_1','item_pv_stair_shop_id_bayes_rate_1','item_brand_id_shop_id_bayes_rate_2','item_id_gender_filled_bayes_rate_1','item_id_gender_filled_bayes_rate_2','shop_id_gender_filled_bayes_rate_1','shop_id_gender_filled_bayes_rate_2','item_id_user_star_stair_bayes_rate_1','shop_id_user_star_stair_bayes_rate_1','shop_id_user_star_stair_bayes_rate_2','item_collected_level_shop_id_bayes_rate_1','item_collected_level_shop_id_bayes_rate_2','user_query_max_score7_percent','item_sales_level_shop_id_bayes_rate_1','item_sales_level_shop_id_bayes_rate_2','item_pv_level_shop_id_bayes_rate_1','user_query_max_score3_percent','user_query_max_score6_percent','user_query_max_score8_percent','shop_id_user_occupation_stair_bayes_rate_2','shop_id_user_occupation_stair_bayes_rate_1','user_query_max_score5_percent','shop_id_user_age_stair_bayes_rate_2','user_query_max_score1_percent','user_query_max_score2_percent','context_page_stair_shop_id_bayes_rate_2','context_page_stair_shop_id_bayes_rate_1','shop_id_user_age_stair_bayes_rate_1','item_id_user_occupation_stair_bayes_rate_2','item_id_user_gender_id_bayes_rate_1','context_page_stair_item_id_bayes_rate_1','context_page_stair_item_id_bayes_rate_2','shop_id_user_gender_id_bayes_rate_2','item_id_user_occupation_stair_bayes_rate_1','item_id_user_gender_id_bayes_rate_2','shop_id_user_gender_id_bayes_rate_1','item_id_user_age_stair_bayes_rate_2','item_id_user_age_stair_bayes_rate_1','user_query_max_score9_percent','shop_id_user_occupation_id_bayes_rate_2','shop_id_user_occupation_id_bayes_rate_1','user_query_max_score4_percent','user_total_query_times','item_brand_id_shop_star_level_bayes_rate_2','item_brand_id_shop_review_num_level_bayes_rate_1','item_brand_id_shop_star_level_bayes_rate_1','item_brand_id_shop_review_num_level_bayes_rate_2','item_id_user_occupation_id_bayes_rate_2','item_id_user_occupation_id_bayes_rate_1','item_brand_id_bayes_rate_1','item_brand_id_bayes_rate_2','user_query_queryitem_c_similarity_rank2','user_query_goodreview_rank','item_brand_id_gender_filled_bayes_rate_2','item_brand_id_gender_filled_bayes_rate_1','user_query_shopstar_rank','item_brand_id_user_star_stair_bayes_rate_1','user_query_service_rank','user_query_queryitem_c_similarity_rank','user_query_review_rank','item_brand_id_user_star_stair_bayes_rate_2','user_query_queryitem_p_similarity_rank2','user_query_delivery_rank','user_query_queryitem_p_similarity_rank','item_brand_id_user_occupation_stair_bayes_rate_2','shop_id_user_age_level_bayes_rate_2','item_brand_id_user_occupation_stair_bayes_rate_1','user_query_pv_rank','user_query_max_score10_rank','user_query_max_score9_rank','context_page_id_shop_id_bayes_rate_2','item_brand_id_user_age_stair_bayes_rate_1','item_brand_id_user_age_stair_bayes_rate_2','item_brand_id_user_gender_id_bayes_rate_1','item_brand_id_user_gender_id_bayes_rate_2','context_page_id_shop_id_bayes_rate_1','context_page_stair_item_brand_id_bayes_rate_2','context_page_id_item_id_bayes_rate_2','context_page_id_item_id_bayes_rate_1','item_brand_id_user_occupation_id_bayes_rate_1','user_query_collect_rank','user_query_price_rank','item_brand_id_user_occupation_id_bayes_rate_2','item_id_user_age_level_bayes_rate_2','item_id_user_age_level_bayes_rate_1','user_current_hour_query','user_query_sales_div_pv_percent','user_query_max_score2','user_query_max_score4_rank','user_query_collect_percent','user_query_price_percent','user_query_max_score3_rank','user_query_max_score8_rank','user_query_max_score6_rank','user_query_queryitem_p_similarity_percent','user_query_queryitem_p_similarity_percent2','shop_id_user_star_level_bayes_rate_1','shop_id_user_star_level_bayes_rate_2','user_today_query','user_today_query_percent','user_query_sale_rank','user_query_queryitem_c_similarity_percent2','user_query_sales_div_pv_rank','user_query_max_score5','user_query_pv_percent','user_query_shopstar_percent','item_id_user_star_level_bayes_rate_2','user_query_max_score7_rank','user_id_bayes_rate_2','item_brand_id_user_age_level_bayes_rate_1','item_id_user_star_level_bayes_rate_1','user_query_max_score1','item_brand_id_user_age_level_bayes_rate_2','user_query_queryitem_c_similarity_percent','user_query_review_percent','user_current_hour_percent','hour_shop_id_bayes_rate_1','user_id_bayes_rate_1','user_query_goodreview_percent','user_query_max_score3','user_query_max_score8','user_query_description_percent','context_page_id_item_brand_id_bayes_rate_1','context_page_id_item_brand_id_bayes_rate_2','item_pv_stair_user_id_bayes_rate_2','item_pv_stair_user_id_bayes_rate_1','user_query_max_score5_rank','user_query_service_percent','time_slice_user_gender_id_bayes_rate_2','user_query_max_score1_rank','user_query_max_score2_rank','time_slice_gender_filled_bayes_rate_1','time_slice_item_price_stair_bayes_rate_2','time_slice_user_gender_id_bayes_rate_1','time_slice_item_price_stair_bayes_rate_1','time_slice','time_slice_gender_filled_bayes_rate_2','hour_gender_filled_bayes_rate_2','hour_user_gender_id_bayes_rate_1','hour_gender_filled_bayes_rate_1','hour_bayes_rate_1','hour_item_pv_stair_bayes_rate_1','time_slice_user_age_level_bayes_rate_1','time_slice_bayes_rate_1','time_slice_bayes_rate_2','time_slice_item_pv_stair_bayes_rate_1','time_slice_item_pv_stair_bayes_rate_2','user_query_max_score6','hour_user_gender_id_bayes_rate_2','time_slice_user_age_stair_bayes_rate_1','hour','time_slice_user_age_stair_bayes_rate_2','time_slice_user_occupation_stair_bayes_rate_2','hour_bayes_rate_2','hour_item_price_stair_bayes_rate_1','hour_item_price_stair_bayes_rate_2','hour_item_pv_stair_bayes_rate_2','hour_user_age_stair_bayes_rate_1','time_slice_user_age_level_bayes_rate_2','time_slice_user_occupation_stair_bayes_rate_1','time_slice_user_star_stair_bayes_rate_2','hour_user_age_level_bayes_rate_1','item_brand_id_user_star_level_bayes_rate_2','item_brand_id_user_star_level_bayes_rate_1','time_slice_user_star_stair_bayes_rate_1','hour_item_brand_id_bayes_rate_2','time_slice_item_sales_stair_bayes_rate_1','hour_user_age_stair_bayes_rate_2','hour_user_star_stair_bayes_rate_2','time_slice_user_occupation_id_bayes_rate_2','hour_user_age_level_bayes_rate_2','hour_user_occupation_id_bayes_rate_2','time_slice_user_occupation_id_bayes_rate_1','hour_user_star_stair_bayes_rate_1','time_slice_item_sales_stair_bayes_rate_2','hour_item_id_bayes_rate_2','hour_item_brand_id_bayes_rate_1','hour_item_city_id_bayes_rate_2','hour_item_sales_stair_bayes_rate_1','hour_user_occupation_id_bayes_rate_1','hour_user_occupation_stair_bayes_rate_1','hour_item_city_id_bayes_rate_1','hour_user_occupation_stair_bayes_rate_2','hour_item_pv_level_bayes_rate_1','hour_item_id_bayes_rate_1','time_slice_item_price_level_bayes_rate_1','time_slice_item_price_level_bayes_rate_2','context_page_stair_user_id_bayes_rate_2','time_slice_item_sales_level_bayes_rate_1','time_slice_item_sales_level_bayes_rate_2','hour_item_sales_level_bayes_rate_1','hour_item_pv_level_bayes_rate_2','context_page_stair_user_id_bayes_rate_1','shop_today_query','time_slice_item_pv_level_bayes_rate_2','hour_user_star_level_bayes_rate_2','hour_user_star_level_bayes_rate_1','item_city_id_shop_star_level_bayes_rate_1','item_price_stair_user_id_bayes_rate_1','time_slice_item_collected_level_bayes_rate_2','time_slice_item_collected_level_bayes_rate_1','item_city_id_shop_star_level_bayes_rate_2','time_slice_item_pv_level_bayes_rate_1','hour_shop_review_num_level_bayes_rate_2','user_query_max_score9','time_slice_user_star_level_bayes_rate_1','hour_item_collected_level_bayes_rate_2','item_price_stair_user_id_bayes_rate_2','hour_shop_star_level_bayes_rate_1','item_city_id_shop_review_num_level_bayes_rate_1','hour_shop_star_level_bayes_rate_2','time_slice_item_city_id_bayes_rate_1','time_slice_shop_id_bayes_rate_2','time_slice_user_star_level_bayes_rate_2','item_city_id_shop_review_num_level_bayes_rate_2','hour_shop_review_num_level_bayes_rate_1','time_slice_item_city_id_bayes_rate_2','user_query_max_score7','shop_today_query_percent','time_slice_shop_id_bayes_rate_1','time_slice_shop_star_level_bayes_rate_2','time_slice_shop_review_num_level_bayes_rate_2','time_slice_shop_star_level_bayes_rate_1','user_query_maxreview','shop_current_hour_query','user_query_best_service','time_slice_shop_review_num_level_bayes_rate_1','user_query_maxshopstar','user_query_best_description','shop_cnt_dayavg_total','time_slice_item_brand_id_bayes_rate_2','user_query_best_delivery','time_slice_item_brand_id_bayes_rate_1','user_cnt_total','user_cnt_dayavg_total','category_1','shop_cnt_total','user_query_maxgoodreview','user_query_max_score4','shop_cnt_day','context_page_id_user_id_bayes_rate_2','item_sales_stair_user_id_bayes_rate_2','item_sales_stair_user_id_bayes_rate_1','context_page_id_user_id_bayes_rate_1','shop_current_hour_percent','user_query_maxcollect','query_category_0','item_price_category_percent','item_city_id_bayes_rate_2','user_query_maxpv','item_city_id_bayes_rate_1','time_slice_item_id_bayes_rate_1','item_city_id_gender_filled_bayes_rate_2','item_city_id_gender_filled_bayes_rate_1','item_city_id_user_star_stair_bayes_rate_2','item_city_id_user_star_stair_bayes_rate_1','brand_today_query','time_slice_item_id_bayes_rate_2','item_city_id_user_gender_id_bayes_rate_1','item_city_id_user_age_stair_bayes_rate_1','item_city_id_user_gender_id_bayes_rate_2','item_city_id_user_age_stair_bayes_rate_2','context_page_stair_item_city_id_bayes_rate_1','item_city_id_user_occupation_id_bayes_rate_1','brand_current_hour_percent','item_price_level_user_id_bayes_rate_2','item_city_id_user_occupation_id_bayes_rate_2','brand_current_hour_query','item_city_id_user_occupation_stair_bayes_rate_2','user_query_max_score10','item_price_category_rank','context_page_stair_item_city_id_bayes_rate_2','item_city_id_user_occupation_stair_bayes_rate_1','user_query_maxsell','item_city_id_user_age_level_bayes_rate_1','item_city_id_user_age_level_bayes_rate_2','item_city_id','user_query_maxsales_div_pv','item_city_sales_category_percent','item_city_id_user_star_level_bayes_rate_1','context_page_id_item_city_id_bayes_rate_1','item_city_id_user_star_level_bayes_rate_2','context_page_id_item_city_id_bayes_rate_2','user_query_maxqueryitem_c_similarity','property_count','category_2','item_brand_price_category_percent','item_brand_pv_category_rank','user_query_maxqueryitem_p_similarity','item_collected_level_user_id_bayes_rate_1','user_query_maxqueryitem_c_similarity2','property_0','item_city_pv_category_percent','item_city_price_category_percent','shop_star_level_user_id_bayes_rate_1','shop_star_level_user_id_bayes_rate_2','item_collected_level_user_id_bayes_rate_2','item_pv_level_user_id_bayes_rate_2','item_city_sales_div_pv_category_percent','item_pv_level_user_id_bayes_rate_1','property_27','item_city_collect_category_rank','property_26','shop_review_num_level_user_id_bayes_rate_1','shop_id_user_gender_id_percent','property_30','user_query_maxqueryitem_p_similarity2','property_29','item_collect_category_rank','item_city_collect_category_percent','property_28','item_brand_sales_category_rank','item_city_sales_category_rank','shop_review_num_level_user_id_bayes_rate_2','item_sales_level_user_id_bayes_rate_2','property_31','item_id_user_gender_id_percent','hour_user_id_bayes_rate_1','item_city_sales_div_pv_category_rank','user_query_cheapest','hour_user_id_bayes_rate_2','item_id_user_id_bayes_rate_1','item_brand_collect_category_rank','shop_avg_age','item_brand_sales_category_percent','item_brand_id_user_gender_id_percent','item_city_pv_category_rank','item_brand_sales_div_pv_category_rank','item_sales_level_user_id_bayes_rate_1','property_33','brand_avg_star','common_category_prob2','item_brand_price_category_rank','user_isnew','item_collect_category_percent','property_32','category_col_47','property_1','property_25','item_id_user_id_bayes_rate_2','item_sales_category_percent','brand_avg_age','property_36','shop_id','property_34','property_38','query_category_1','property_3','shop_id_user_id_bayes_rate_1','shop_avg_star','property_24','item_brand_collect_category_percent','item_sales_div_pv_rank','item_brand_sales_div_pv_category_percent','property_2','item_pv_category_rank','property_35','item_pv_category_percent','category_col_19','shop_id_user_id_bayes_rate_2','query_col_0','property_39','item_price_level_user_age_stair_bayes_rate_1','item_price_level_user_age_stair_bayes_rate_2','item_sales_category_rank','common_category','category_col_78','item_brand_pv_category_percent','common_category_prob','item_current_hour_query','shop_id_user_age_level_percent','property_22','property_6','property_12','item_price_level_bayes_rate_1','item_price_level_bayes_rate_2','shop_id_user_occupation_id_percent','item_avg_age','property_37','category_col_31','item_price_level_user_gender_id_bayes_rate_1','item_price_level_user_gender_id_bayes_rate_2','property_11','property_4','item_avg_star','user_cnt_day','item_price_stair_user_age_level_bayes_rate_2','item_price_level_user_star_stair_bayes_rate_1','item_price_level_user_star_stair_bayes_rate_2','property_19','user_shop_age_dist','item_price_stair_user_age_level_bayes_rate_1','item_price_stair_user_age_stair_bayes_rate_1','item_price_stair_user_age_stair_bayes_rate_2','item_collected_level_shop_review_num_level_bayes_rate_1','user_gender_id_bayes_rate_1','user_gender_id_bayes_rate_2','item_price_level_user_age_level_bayes_rate_1','property_10','context_page_id_user_age_stair_bayes_rate_2','item_collected_level_shop_review_num_level_bayes_rate_2','item_price_stair_user_occupation_id_bayes_rate_1','item_price_stair_user_occupation_id_bayes_rate_2','item_price_stair_gender_filled_bayes_rate_2','property_14','item_price_stair_gender_filled_bayes_rate_1','item_today_query','property_7','category_col_56','item_city_id_user_id_bayes_rate_2','item_price_level_gender_filled_bayes_rate_1','item_price_level_gender_filled_bayes_rate_2','user_brand_age_dist','category_col_84','item_pv_stair_gender_filled_bayes_rate_2','user_brand_star_dist','user_shop_dist_dist','item_brand_id','category_col_45','item_pv_stair_user_star_stair_bayes_rate_1','item_pv_stair_user_star_stair_bayes_rate_2','shop_review_num_level_bayes_rate_1','context_page_id_gender_filled_bayes_rate_1','context_page_id_user_star_stair_bayes_rate_2','item_price_stair_user_star_stair_bayes_rate_1','item_price_stair_user_star_stair_bayes_rate_2','item_collected_level_shop_star_level_bayes_rate_1','item_sales_level_user_gender_id_bayes_rate_2','category_col_59','item_collected_level_shop_star_level_bayes_rate_2','item_price_stair_bayes_rate_1','item_price_stair_bayes_rate_2','item_sales_level_user_gender_id_bayes_rate_1','property_8','time_slice_user_id_bayes_rate_2','item_cnt_dayavg_total','property_9','category_col_83']
features_cjf.remove('hour')
features_cjf.remove('user_cnt_day')
features_cjf.remove('shop_cnt_day')

features_cjy = ['item_cat_vec0', 'item_cat_vec1', 'item_cat_vec2',
       'item_cat_vec3', 'item_cat_vec4', 'item_cat_vec5', 'item_cat_vec6',
       'item_cat_vec7', 'item_cat_vec8', 'item_cat_vec9', 'item_cat_vec10',
       'item_cat_vec11', 'item_cat_vec12', 'item_cat_vec13', 'item_cat_vec14',
       'item_cat_vec15', 'shop_item_cnt', 'shop_brand_cnt',
       'item_pred_cat_cos', 'item_pred_ppt_cos', 'age_category_min',
       'gender_category_mean', 'occupation_category_max', 'star_category_max',
       'gender_category_min', 'star_category_min', 'occupation_category_min',
       '24h_cat_hot_item', '24h_ppt_hot_item', '24h_user_seem_times_item',
       '24h_cat_hot', '24h_ppt_hot', '24h_user_seem_times', 'item_id_pred_1',
       'shop_id_pred_1','item_brand_id_pred_1','user_id_pred_1','context_hour_pred_1',
       'hist_hour_ctr_pred_1','shop_item_cnt_pred_1','gender_category_min_pred_1','gender_category_mean_pred_1',
       'occupation_category_min_pred_1','item_brand_id+item_city_id_pred_1','item_brand_id+item_price_level_pred_1',
       'item_brand_id+item_sales_level_pred_1','item_brand_id+item_collected_level_pred_1','item_brand_id+item_pv_level_pred_1',
       'context_page_id+context_hour_pred_1','shop_review_num_level+shop_item_cnt_pred_1','shop_star_level+shop_item_cnt_pred_1',
       'shop_item_cnt+shop_brand_cnt_pred_1','shop_review_positive_rate+shop_item_cnt_pred_1','shop_review_num_level+shop_review_positive_rate_pred_1',
       'item_city_id+item_pv_level_pred_1','item_sales_level+item_collected_level_pred_1','item_city_id+item_price_level_pred_1',
       'shop_score_description+shop_item_cnt_pred_1','shop_score_delivery+shop_item_cnt_pred_1','shop_review_positive_rate+shop_star_level_pred_1',
       'item_price_level+item_collected_level_pred_1','shop_score_service+shop_score_delivery_pred_1','shop_star_level+shop_brand_cnt_pred_1',
       'item_city_id+item_collected_level_pred_1','item_city_id+item_sales_level_pred_1','shop_star_level+shop_score_description_pred_1',
       'shop_review_positive_rate+shop_score_service_pred_1','user_age_level+user_star_level_pred_1','shop_review_num_level+shop_brand_cnt_pred_1',
       'shop_score_service+shop_score_description_pred_1','shop_review_num_level+shop_score_service_pred_1','shop_star_level+shop_score_delivery_pred_1',
       'user_occupation_id+user_star_level_pred_1']

target = ['is_trade']

by = ['instance_id']

In [4]:
predictors = list(set(origin + features_zero + features_cjf + features_cjy))
len(predictors)

637

## 读数据

In [5]:
train = pd.read_csv('train_model2_mix_final.csv')
test = pd.read_csv('test_model2_mix_final.csv')

train.shape
test.shape

(2357860, 642)

(1729656, 642)

## 切分结果

In [6]:
split_dict = pickle.load(open('45th_5f_stratifiedkfold.pk', 'rb'))
split_dict

{0: array([      1467332744642,      31168815605908,      34184420712504, ...,
        5971377741080961986, 5974120441804431573, 5984460737438686294]),
 1: array([4013803974121919208, 4013915255555117537, 4014097568838562892, ...,
        5464551454004330330, 5466156937331656891, 5470385729918491039]),
 2: array([ 239711996433822837,  239734248409520136,  239775700966388495, ...,
        8573050094989475734, 8578725633646746836, 8579438100404380058]),
 3: array([4154201336437765114, 4154375151458362143, 4154380146478054260, ...,
        8266179397285344615, 8266755394251995579, 8267168646953887462]),
 4: array([6060183997054029599, 6060304206666061504, 6060435031736288732, ...,
        9223078109497825671, 9223185444724062390, 9223340447851431399])}

In [7]:
train[train.instance_id.isin(list(split_dict[0]))].shape

(471573, 642)

## 训练

In [8]:
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier

In [9]:
def stacking_day(d_train, d_test, d):
    train = d_train[~d_train.instance_id.isin(list(split_dict[d]))]
    pred  = d_train[d_train.instance_id.isin(list(split_dict[d]))]
    return model_train(train, pred, d_test)

In [10]:
model_name = 'lgb_zero'
def model_train(train, pred, test):   
    # 参数
    lgb_online = lgb.LGBMClassifier(
        objective='binary',
        metric='binary_logloss',
        num_leaves=7,#35,
        max_depth=3,
        learning_rate=0.05, # small_eta
        reg_alpha = .4,
        reg_lambda = .2,
        colsample_bytree = .8,
        subsample = .9,
        random_state=1024,
        n_estimators=1400,
        n_jobs = 31,
        histogram_pool_size=30270,
        max_bin=63
    )
    # 训练模型
    model = lgb_online.fit(train[predictors],train['is_trade'],eval_set=[(pred[predictors],pred['is_trade'])],early_stopping_rounds=50,verbose=False)
    return model.predict_proba(pred[predictors])[:,1], model.predict_proba(test[predictors])[:,1]

In [11]:
%%time
train[model_name+'_prob'] = 0
for d in range(0, 5):
    pred_prob, test_prob = stacking_day(train, test, d)
    train[model_name+'_prob'][train.instance_id.isin(list(split_dict[0]))] = pred_prob
    test[model_name + '_prob_%d'%d] = test_prob
train[['instance_id',model_name+'_prob']].to_csv('./stacking/train_%s.csv'%(model_name),index=False)
test[model_name + '_prob'] = testset.iloc[:,-4:].mean(axis=1)
test[['instance_id',model_name + '_prob']].to_csv('./stacking/test_%s.csv'%(model_name),index=False)

MemoryError: 

In [12]:
model_name = 'xgb_zero'
def model_train(train, pred, test):   
    # 参数
    params = {
        'objective': 'binary:logistic',
        'eta': 0.05,
        'colsample_bytree': 0.886,
        'min_child_weight': 1.6,
        'max_depth': 4,
        'subsample': 0.886,
        'gamma': 0.1,
        'lambda': 10,
        'verbose_eval': True,
        'eval_metric': 'logloss',
        'seed': 201803,
        'missing': -1
    }

    len(predictors)

    xgbtrain = xgb.DMatrix(train[predictors], train[target])
    xgbtest = xgb.DMatrix(test[predictors])
    model = xgb.train(params, xgbtrain, num_boost_round=cv.shape[0])

    return model.predict(pred[predictors]), model.predict(test[predictors])

In [13]:
%%time
train[model_name+'_prob'] = 0
for d in range(0, 5):
    pred_prob, test_prob = stacking_day(train, test, d)
    train[model_name+'_prob'][train.instance_id.isin(list(split_dict[0]))] = pred_prob
    test[model_name + '_prob_%d'%d] = test_prob
train[['instance_id',model_name+'_prob']].to_csv('./stacking/train_%s.csv'%(model_name),index=False)
test[model_name + '_prob'] = testset.iloc[:,-4:].mean(axis=1)
test[['instance_id',model_name + '_prob']].to_csv('./stacking/test_%s.csv'%(model_name),index=False)

MemoryError: 

In [14]:
model_name = 'rf_zero'
def model_train(train, pred, test):   
    # 参数
    clf = RandomForestClassifier(
    n_estimators=500, 
    max_depth=6, 
    min_samples_leaf=15,
    n_jobs=-1)
    clf.fit(train[predictors], train[target])
    
    return model.predict_proba(pred[predictors])[:,1], model.predict_proba(test[predictors])[:,1]

In [15]:
%%time
train[model_name+'_prob'] = 0
for d in range(0, 5):
    pred_prob, test_prob = stacking_day(train, test, d)
    train[model_name+'_prob'][train.instance_id.isin(list(split_dict[0]))] = pred_prob
    test[model_name + '_prob_%d'%d] = test_prob
train[['instance_id',model_name+'_prob']].to_csv('./stacking/train_%s.csv'%(model_name),index=False)
test[model_name + '_prob'] = testset.iloc[:,-4:].mean(axis=1)
test[['instance_id',model_name + '_prob']].to_csv('./stacking/test_%s.csv'%(model_name),index=False)

MemoryError: 

## stacking

In [ ]:
train1 = pd.read_csv('./stacking/train_lgb_ze.csv')
train2 = pd.read_csv('./stacking/train_xgb_ze.csv')
train3 = pd.read_csv('./stacking/train_rf_ze.csv')

train1.shape
train2.shape
train3.shape

In [ ]:
train = train1.merge(train2, on=['instance_id'], how='left')
train = train.merge(train3, on=['instance_id'], how='left')

del train1
del train2
del train3
gc.collect()